In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, email
import seaborn as sns; sns.set_style('whitegrid')
#import plotly
#plotly.offline.init_notebook_mode()
#import plotly.graph_objs as go
import wordcloud
import networkx as nx
from nltk.tokenize.regexp import RegexpTokenizer
from subprocess import check_output

from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD 
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import linear_kernel
from helpers import parse_into_emails

import pandas as pd

#from helpers import * 

emails = pd.read_csv('emails.csv')
emails.shape 
#emails.head()

(517401, 2)

In [2]:

#print(check_output(["ls", "../desktop/Enron"]).decode("utf8"))
# # 1. Loading and cleaning data
# Read the data into a DataFrame
emails_df = pd.read_csv('../Enron/emails.csv')
print(emails_df.shape)
emails_df.head()
# A single message looks like this
print(emails_df['message'][0])
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs


(517401, 2)
Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [3]:
emails_df = pd.read_csv('../Enron/emails.csv')
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)
# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

#print('shape of the dataframe:', emails_df.shape)

# Find number of unique values in each columns
for col in emails_df.columns:
    print(col, emails_df[col].nunique())
# Set index and drop columns with two few values
emails_df = emails_df.set_index('Message-ID')\
    .drop(['file', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding'], axis=1)
# Parse datetime
emails_df['Date'] = pd.to_datetime(emails_df['Date'], infer_datetime_format=True)
#emails_df.type()
#emails_df.head()


file 517401
Message-ID 517401
Date 224128
From 20328
To 54748
Subject 159290
Mime-Version 1
Content-Type 2
Content-Transfer-Encoding 3
X-From 27980
X-To 73552
X-cc 33701
X-bcc 132
X-Folder 5335
X-Origin 259
X-FileName 429
content 249025
user 150


In [4]:
emails_clean = emails_df.drop(columns=['X-From', 'X-To', 'X-Origin', 'X-FileName', 'X-Folder'])

#emails_clean.describe()

In [5]:
emails_clean = emails_df.drop_duplicates()

In [20]:
#emails_clean.From
import re
m = re.search(r'\w+\.\w+\@e\w+on\.com', "pete.davis@enron.com")


#wrap around for loop


print(m)

<_sre.SRE_Match object; span=(0, 20), match='pete.davis@enron.com'>


In [5]:
sub_df = emails_clean[['From', 'To', 'Date']].dropna()
print(sub_df.shape)
# drop emails sending to multiple addresses
sub_df = sub_df.loc[sub_df['To'].map(len) == 1]
#print(sub_df.shape)
sub_df = sub_df.groupby(['From', 'To']).count().reset_index()
# Unpack frozensets
sub_df['From'] = sub_df['From'].map(lambda x: next(iter(x)))
sub_df['To'] = sub_df['To'].map(lambda x: next(iter(x)))

sub_df.rename(columns={'Date': 'count'}, inplace=True)
sub_df.sort_values('count', ascending=False).head(10)


(495554, 3)
(354600, 3)


,From,To,count
17908,pete.davis@enron.com,pete.davis@enron.com,9141
38033,vince.kaminski@enron.com,vkaminski@aol.com,4308
28920,enron.announcements@enron.com,all.worldwide@enron.com,2206
28935,enron.announcements@enron.com,all.houston@enron.com,1701
26510,kay.mann@enron.com,suzanne.adams@enron.com,1528
38031,vince.kaminski@enron.com,shirley.crenshaw@enron.com,1190
14564,steven.kean@enron.com,maureen.mcvicker@enron.com,1014
26309,kay.mann@enron.com,nmann@erac.com,980
18926,kate.symes@enron.com,evelyn.metoyer@enron.com,915
18930,kate.symes@enron.com,kerri.thompson@enron.com,859


In [ ]:
import spacy
import pickle

In [ ]:
emails_clean.set_index('Date',inplace=True)

In [ ]:
emails_clean = emails_clean['1997':'2002']

In [ ]:
emails_clean.to_csv('Enron.csv')

In [ ]:
emails_clean.resample('M')['To'].count().plot(marker='o')

In [ ]:
emails_clean.index.year.value_counts().sort_index().plot.bar()

In [ ]:
emails_clean.index.month.value_counts().sort_index().plot.bar()

In [ ]:
emails_clean.index.hour.value_counts().sort_index().plot.bar()

In [ ]:
# Top 20 Froms
(lambda x: set(x[:25].index))(emails_clean['From'].value_counts())

In [ ]:
# Top 20 Froms

(lambda x: set(x[:25].index))(emails_clean['To'].value_counts())

In [ ]:
emails_clean.describe()

In [6]:
pd.pivot_table(emails_clean,index=["From"],columns= ["To"], values=["content"], aggfunc='count')


content  \
To                                       (tim.belden@enron.com)   
From                                                              
(phillip.allen@enron.com)                                  16.0   
(sbofni@yahoo.com)                                          NaN   
(zaeza@unplace.com)                                         NaN   
(memories@corp.classmates.com)                              NaN   
(emca@yahoogroups.com)                                      NaN   
(mbrousse@kinkaid.org)                                      NaN   
(benjamin@wt.net)                                           NaN   
(josena@flash.net)                                          NaN   
(kpiotrus@genesiscrudeoil.com)                              NaN   
(rgwelch@insync.net)                                        NaN   
(bungalowlife@yahoo.com)                                    NaN   
(2000greetings@postalmanager.com)                           NaN   
(bobnovotny@juno.com)                                       NaN   
(barnesfamilyus@prodigy.net)                                NaN   
(heddamcallister@aol.com)                                   NaN   
(futureprospects@aol.com)                                   NaN   
(bhastings@earthlink.net)                                   NaN   
(gayle@rodgers-ramsey.com)                                  NaN   
(cpa@brain2002.com)                                         NaN   
(johnb@borrowmanbaker.com)                                  NaN   
(evanscj@swbell.net)                                        NaN   
(myates@paradigmgeo.com)                                    NaN   
(jstelly@pdq.net)                                           NaN   
(onoisycats@aol.com)                                        NaN   
(edgonzales@ev1.net)                                        NaN   
(stephen.veatch@enron.com)                                  NaN   
(welchj@foresightps.com)                                    NaN   
(joni.moeller@enron.com)                                    NaN   
(hsmith3@attglobal.net)                                     NaN   
(joe@joewhite.com)                                          NaN   
...                                                         ...   
(patty@sbccottonbowl.com)                                   NaN   
(sunil.abraham@enron.com)                                   NaN   
(kevin.radous@enron.com)                                    NaN   
(birgit@hiresynergy.com)                                    NaN   
(a@echostar.com)                                            NaN   
(jhliberman@pclient.ml.com)                                 NaN   
(dgiron1@houston.rr.com)                                    NaN   
(deirdre.mccaffrey@enron.com)                               NaN   
(ralphmayers@hotmail.com)                                   NaN   
(zebradesign@libero.it)                                     NaN   
(lucy.ortiz@enron.com)                                      NaN   
(bwells@pop.net)                                            NaN   
(linnierozelle@compaq.net)                                  NaN   
(melf116@aol.com)                                           NaN   
(mary.ruffer@enron.com)                                     NaN   
(onepass_1221@airmail.continental.com)                      NaN   
(eric.wardle@enron.com)                                     NaN   
(onepass_121800@airmail.continental.com)                    NaN   
(ken.sissingh@enron.com)                                    NaN   
(jbrownin@utilicorp.com)                                    NaN   
(antonio.paez@bradfordcompany.com)                          NaN   
(c..kenne@enron.com)                                        NaN   
(mrichter@us.ibm.com)                                       NaN   
(ronniegiron@yahoo.com)                                     NaN   
(robin.rodrigue@enron.com)                                  NaN   
(bernard.dahanayake@enron.com)                              NaN   
(joe.bucio@enron.com)               